In [4]:
import pandas as pd
from GameFeatures import GameFeatures
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv('gamesDB.csv')

In [6]:
df.columns

Index(['Unnamed: 0', 'data', 'time_casa', 'time_visitante', 'resultado',
       'id_partida', 'link'],
      dtype='object')

In [7]:
df.drop(columns='Unnamed: 0', inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null object
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: object(6)
memory usage: 213.8+ KB


In [9]:
df.data[0]

'2007-05-12 19:10:00'

In [10]:
df.data = pd.to_datetime(df.data)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null datetime64[ns]
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 213.8+ KB


In [12]:
mask = df.data.dt.year > 2009

In [14]:
df[mask].head()

,data,time_casa,time_visitante,resultado,id_partida,link
1140,2010-05-08 19:30:00,Botafogo,Santos,3-3,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1141,2010-05-08 19:30:00,Atlético GO,Grêmio,0-0,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1142,2010-05-08 19:35:00,Palmeiras,Vitória,1-0,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1143,2010-05-09 17:00:00,Flamengo,São Paulo,1-1,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1144,2010-05-09 17:00:00,Atlético MG,Vasco,2-1,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...


In [15]:
links = df.link[1140:1150]

In [16]:
home_team = []
away_team = []
home_team_score = []
away_team_score = []
referee = []
stadium = []
city = []
datetime = []
game = []
eventList1st = []
eventList2nd = []
lineupList = []
sublineupList = []
statsHome = []
statsAway = []

In [17]:
i = 0
games_len = len(links)
for url in links:
    home_teamT, away_teamT, home_team_scoreT, away_team_scoreT, refereeT, stadiumT, cityT, datetimeT, gameT, eventList1stT, eventList2ndT, lineupListT, sublineupListT, statsHomeT, statsAwayT = GameFeatures(url)
    
    home_team.append(home_teamT) 
    away_team.append(away_teamT) 
    home_team_score.append(home_team_scoreT) 
    away_team_score.append(away_team_scoreT) 
    referee.append(refereeT) 
    stadium.append(stadiumT) 
    city.append(cityT) 
    datetime.append(datetimeT) 
    game.append(gameT) 
    eventList1st.append(eventList1stT) 
    eventList2nd.append(eventList2ndT) 
    lineupList.append(lineupListT) 
    sublineupList.append(sublineupListT) 
    statsHome.append(statsHomeT) 
    statsAway.append(statsAwayT)
    
    i+=1
    print('Game number: '+str(i)+'/'+str(games_len))

Game number: 1/10
Game number: 2/10
Game number: 3/10
Game number: 4/10
Game number: 5/10
Game number: 6/10
Game number: 7/10
Game number: 8/10
Game number: 9/10
Game number: 10/10


In [19]:
df = pd.DataFrame({'home_team':home_team, 
                    'away_team':away_team, 
                    'home_team_score':home_team_score, 
                    'away_team_score':away_team_score, 
                    'referee':referee, 
                    'stadium':stadium, 
                    'city':city, 
                    'datetime':datetime, 
                    'game':game, 
                    'eventList1st':eventList1st, 
                    'eventList2nd':eventList2nd, 
                    'lineupList':lineupList, 
                    'sublineupList':sublineupList, 
                    'statsHome':statsHome, 
                    'statsAway':statsAway})

In [20]:
df.to_csv('soccerDB.csv')